In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from src.stereo_inference import StereoInferenceOnnx

In [ ]:
#load images
left_image = cv2.imread("data/samples/seat_l_n.png", cv2.IMREAD_COLOR)
right_image = cv2.imread("data/samples/seat_r_n.png", cv2.IMREAD_COLOR)
# Convert from BGR to RGB
left_image_rgb = cv2.cvtColor(left_image, cv2.COLOR_BGR2RGB)
right_image_rgb = cv2.cvtColor(right_image, cv2.COLOR_BGR2RGB)

In [ ]:
# Display the stereo pair
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.imshow(left_image_rgb)
plt.title('Left Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(right_image_rgb)
plt.title('Right Image')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Initialize the ONNX model
model_path = "models/fs_768_1024.onnx"
inference_engine = StereoInferenceOnnx(model_path)

In [ ]:
# Function to preprocess images for the model
def preprocess_images(left_img, right_img, target_height=None, target_width=None):
    """
    Preprocess images for ONNX inference:
    1. Resize if target dimensions are provided
    2. Convert to RGB if needed
    3. Normalize to float32 [0-255]
    4. Transpose from HWC to CHW format
    5. Add batch dimension
    """
    # Resize if dimensions are provided
    if target_height is not None and target_width is not None:
        left_img = cv2.resize(left_img, (target_width, target_height))
        right_img = cv2.resize(right_img, (target_width, target_height))
    
    # Get current dimensions
    height, width = left_img.shape[:2]
    
    # Convert to RGB if in BGR format
    if left_img.shape[2] == 3:  # If color image
        left_img = cv2.cvtColor(left_img, cv2.COLOR_BGR2RGB)
        right_img = cv2.cvtColor(right_img, cv2.COLOR_BGR2RGB)
    
    # Convert to float32 [0-255]
    left_img = left_img.astype(np.float32)
    right_img = right_img.astype(np.float32)
    
    # Transpose from HWC to CHW format
    left_img = left_img.transpose(2, 0, 1)  # (C, H, W)
    right_img = right_img.transpose(2, 0, 1)  # (C, H, W)
    
    # Add batch dimension
    left_img = np.expand_dims(left_img, axis=0)  # (1, C, H, W)
    right_img = np.expand_dims(right_img, axis=0)  # (1, C, H, W)
    
    return left_img, right_img, height, width

In [ ]:
target_height = 1024
target_width = 768

left_tensor, right_tensor, orig_height, orig_width = preprocess_images(
    left_image, right_image, target_height, target_width
)

print(f"Input tensor shapes: {left_tensor.shape}, {right_tensor.shape}")

In [ ]:
disparity = inference_engine(left_tensor, right_tensor)
print(f"Disparity output shape: {disparity.shape}")

In [ ]:
def visualize_disparity(disparity_map, min_disp=None, max_disp=None):
    """
    Visualize disparity map with optional min/max normalization.
    Returns a colorized disparity map.
    """
    # Remove batch dimension if present
    if disparity_map.ndim == 4:
        disparity_map = disparity_map[0]
    if disparity_map.ndim == 3:
        disparity_map = disparity_map[0]  # Keep only the first channel
    
    # Handle NaN and Inf values
    disparity_map = np.nan_to_num(disparity_map, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Normalize disparity to 0-1 range for visualization
    if min_disp is None:
        min_disp = np.min(disparity_map)
    if max_disp is None:
        max_disp = np.max(disparity_map)
    
    # Clip values and normalize
    normalized_disparity = np.clip(disparity_map, min_disp, max_disp)
    normalized_disparity = (normalized_disparity - min_disp) / (max_disp - min_disp)
    
    # Apply colormap
    colored_disparity = cv2.applyColorMap((normalized_disparity * 255).astype(np.uint8), 
                                          cv2.COLORMAP_TURBO)
    colored_disparity = cv2.cvtColor(colored_disparity, cv2.COLOR_BGR2RGB)
    
    return colored_disparity, normalized_disparity

In [ ]:
# Visualize the disparity map
colored_disparity, normalized_disparity = visualize_disparity(disparity)

In [ ]:
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.imshow(left_image_rgb)
plt.title('Original Left Image')
plt.axis('off')

plt.subplot(2, 1, 2)
plt.imshow(colored_disparity)
plt.title('Disparity Map')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Import Open3D for point cloud processing
import open3d as o3d

# Helper functions from the original implementation
def depth2xyzmap(depth, K):
    """Convert depth map to 3D point cloud in camera coordinates."""
    h, w = depth.shape
    y, x = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')
    
    # Convert image coordinates to normalized device coordinates
    x = (x - K[0, 2]) / K[0, 0]
    y = (y - K[1, 2]) / K[1, 1]
    
    # Create homogeneous coordinates
    xyz = np.stack([x, y, np.ones_like(depth)], axis=-1)
    
    # Scale by depth
    xyz = xyz * depth[..., np.newaxis]
    
    return xyz

def toOpen3dCloud(xyz, rgb=None):
    """Convert point cloud data to Open3D format."""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    
    if rgb is not None:
        # Normalize RGB to [0, 1]
        if rgb.max() > 1:
            rgb = rgb / 255.0
        pcd.colors = o3d.utility.Vector3dVector(rgb)
    
    return pcd

# %%
# Generate and visualize point cloud
def generate_point_cloud(disparity_map, intrinsic_file, scale=1.0, z_far=10.0, 
                        denoise_cloud=True, denoise_nb_points=30, denoise_radius=0.03,
                        output_dir="output/"):
    """
    Generate point cloud from disparity map using camera intrinsics.
    
    Args:
        disparity_map: Disparity map as numpy array
        intrinsic_file: Path to file containing camera intrinsics
        scale: Scale factor (if images were resized)
        z_far: Maximum depth to clip in point cloud
        denoise_cloud: Whether to denoise the point cloud
        denoise_nb_points: Number of points to consider for radius outlier removal
        denoise_radius: Radius to use for outlier removal
        output_dir: Directory to save results
    
    Returns:
        Point cloud object
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    # Remove batch dimension if present
    if disparity_map.ndim == 4:
        disparity_map = disparity_map[0, 0]
    elif disparity_map.ndim == 3:
        disparity_map = disparity_map[0]
    
    # Load camera intrinsics
    with open(intrinsic_file, 'r') as f:
        lines = f.readlines()
        K = np.array(list(map(float, lines[0].rstrip().split()))).astype(np.float32).reshape(3, 3)
        baseline = float(lines[1])
    
    # Scale intrinsics if images were resized
    K[:2] *= scale
    
    # Calculate depth from disparity
    # Avoid division by zero
    disparity_map = np.maximum(disparity_map, 0.1)
    depth = K[0, 0] * baseline / disparity_map
    
    # Save depth map
    np.save(f'{output_dir}/depth_meter.npy', depth)
    
    # Use original RGB image for coloring
    img_rgb = left_image_rgb
    
    # Convert depth to 3D points
    xyz_map = depth2xyzmap(depth, K)
    
    # Create point cloud
    pcd = toOpen3dCloud(xyz_map.reshape(-1, 3), img_rgb.reshape(-1, 3))
    
    # Filter points by depth range
    keep_mask = (np.asarray(pcd.points)[:, 2] > 0) & (np.asarray(pcd.points)[:, 2] <= z_far)
    keep_ids = np.arange(len(np.asarray(pcd.points)))[keep_mask]
    pcd = pcd.select_by_index(keep_ids)
    
    # Save point cloud
    o3d.io.write_point_cloud(f'{output_dir}/cloud.ply', pcd)
    print(f"Point cloud saved to {output_dir}/cloud.ply")
    
    # Denoise point cloud if requested
    if denoise_cloud:
        print("Denoising point cloud...")
        cl, ind = pcd.remove_radius_outlier(nb_points=denoise_nb_points, radius=denoise_radius)
        inlier_cloud = pcd.select_by_index(ind)
        o3d.io.write_point_cloud(f'{output_dir}/cloud_denoise.ply', inlier_cloud)
        print(f"Denoised point cloud saved to {output_dir}/cloud_denoise.ply")
        pcd = inlier_cloud
    
    return pcd

In [ ]:
intrinsic_file = "camera_configs/K.txt"  # Path to your intrinsic file
scale = 1/4  # Set to the scale factor if images were resized
z_far = 5  # Maximum depth in meters

# Generate and visualize point cloud
pcd = generate_point_cloud(
    disparity_map=disparity,
    intrinsic_file=intrinsic_file,
    scale=scale,
    z_far=z_far,
    denoise_cloud=True,
    denoise_nb_points=30,
    denoise_radius=0.03,
    output_dir="output/"
)

# Visualize point cloud
print("Visualizing point cloud. Press ESC to exit.")
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.get_render_option().point_size = 1.0
vis.get_render_option().background_color = np.array([0.5, 0.5, 0.5])
vis.run()
vis.destroy_window()